# Langchain bases

# Summary

- Setup model, 
- Langchain resources, 
- LCEL (Langchain Expression Language) 
- Prompt template
- Output parser
- Parallelism (chains)
- Quick RAG search example
- Tools 
- Agents
- Agents type and agents events 
- Raising an agent
- Memory 

# Setup Langchain

In [ ]:
%pip install --upgrade --quiet  langchain-core langchain-community langchain-openai langchain docarray tiktoken langchainhub

In [1]:
import openai
import json

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.memory import ChatMessageHistory
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.messages import HumanMessage, AIMessage

# Setup model

In [2]:
with open('../credentials.json') as jsonfile:
    credentials = json.load(jsonfile)

In [3]:
with open('../embeddings.json') as jsonfile:
    embeddings = json.load(jsonfile)

In [4]:
model = AzureChatOpenAI(
                        azure_endpoint = credentials['OpenAI']['base'],
                        openai_api_version = credentials['OpenAI']['version'],
                        openai_api_key = credentials['OpenAI']['key'],
                        openai_api_type = credentials['OpenAI']['type'],
                        deployment_name = credentials['OpenAI']['deployment_name'],
                        temperature = 0.0)

In [5]:
embeddings = AzureOpenAIEmbeddings(
                        azure_endpoint = embeddings['embeddings']['base'],
                        openai_api_version = embeddings['embeddings']['version'],
                        openai_api_key = embeddings['embeddings']['key'],
                        openai_api_type = embeddings['embeddings']['type'],
                        azure_deployment = embeddings['embeddings']['deployment_name'])

# Langchain resources

## Currently

- ``langchain-core``: Base abstractions and LangChain Expression Language.
- ``langchain-community``: Third party integrations.
    - Partner packages (e.g. langchain-openai, langchain-anthropic, etc.): Some integrations have been further split into their own lightweight packages that only depend on langchain-core.
- ``langchain``: Chains, agents, and retrieval strategies that make up an application's cognitive architecture.
- ``langgraph``: Build robust and stateful multi-actor applications with LLMs by modeling steps as edges and nodes in a graph.
- ``langserve``: Deploy LangChain chains as REST APIs.

![](https://python.langchain.com/svg/langchain_stack.svg)

# How langchain works

<center><img src="https://raw.githubusercontent.com/davidmigloz/langchain_dart/main/docs/img/langchain.dart.png
" width="1000"></center>

# How prompt templates work

## LCEL: Basic example: prompt + model + output parser

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [ ]:
chain.invoke({"topic": "ice cream"})

In [ ]:
# https://python.langchain.com/docs/modules/model_io/output_parsers/

## Prompt template + tools testing

In [22]:
# set a prompt template 

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
system = prompt_template.format(adjective="funny", content="chickens")

In [20]:
@tool
def chain_testing_prompt_tools(input_text) -> str:
    """
    Chain to tell a very funny joke
    """
    prompt = ChatPromptTemplate.from_messages([
            ("system", system),
            ("user", "{input}")
        ])

    chain = prompt | model 

    return chain.invoke({"input": input_text})

In [ ]:
chain_testing_prompt_tools.invoke("Tell me a joke about chickens")

## Pydantic + output parser

In [ ]:
# https://docs.pydantic.dev/latest/

In [ ]:
# define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # you can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [ ]:
# set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
# and a query intended to prompt a language model to populate the data structur

prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

## Parallelism

In [ ]:
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}") | model)

In [ ]:
combined_chains = RunnableParallel(joke=joke_chain,bears=poem_chain)

# we can invoke the runnable normally using the invoke method
combined_chains.invoke({"topic":"bears"})

### Parallelism can also be used with batches

In [ ]:
combined_chains.batch([{"topic": "bears"}, {"topic": "cats"}])

# RAG Search Example

In [ ]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=embeddings,
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

In [ ]:
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

# How Langchain tools work

In [ ]:
# define a tool using the '@tool' decorator. 
# this tool will perform a multiplication operation.
@tool
def multiply(first_int: int, second_int: int) -> int:
    """
    This function multiplies two integers together and returns the result.
    
    Parameters:
    first_int (int): The first integer to be multiplied.
    second_int (int): The second integer to be multiplied.

    Returns:
    int: The product of the two input integers.
    """
    # perform the multiplication operation and return the result.
    return first_int * second_int

- the '@tool' decorator is used to indicate that you want to create your own tool

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

In [ ]:
multiply.invoke({"first_int": 4, "second_int": 5})

# How Langchain agents work

<center><img src="https://miro.medium.com/v2/resize:fit:1358/1*5TnpUZnp4-sq8TuJGYe_-w.png" width="1300"></center>

- Agents are a crucial component of Langchain. 
- They are tasked with making decisions and carrying out actions based on a language model. 
- Unlike action chains, where a sequence of actions is directly hardcoded into the code, agents employ a language model as a reasoning mechanism to determine which actions to take and in what order.

## Built-in LangChain tools

## Agent types

- ``CHAT_CONVERSATIONAL_REACT_DESCRIPTION``: Este agente é projetado para ser usado em configurações de conversação. Ele usa o framework ReAct para decidir qual ferramenta usar e usa a memória para lembrar as interações anteriores da conversa.

- ``CHAT_ZERO_SHOT_REACT_DESCRIPTION``: Este agente é uma versão otimizada para modelos de chat. Ele usa o framework ReAct para decidir qual ferramenta usar e pode invocar ferramentas com várias entradas.

- ``CONVERSATIONAL_REACT_DESCRIPTION``: Este agente usa o framework ReAct para decidir qual ferramenta usar com base na descrição da ferramenta. Ele pode receber várias ferramentas como entrada.

- ``OPENAI_FUNCTIONS``: Este agente é otimizado para usar funções específicas do OpenAI. Ele é projetado para trabalhar com modelos que foram ajustados para detectar quando uma função deve ser chamada e responder com as entradas que devem ser passadas para a função.

- ``OPENAI_MULTI_FUNCTIONS``: Este agente é uma versão estendida do OPENAI_FUNCTIONS e suporta a chamada de várias funções ao mesmo tempo. Isso pode acelerar a execução de agentes em certas tarefas.

- ``REACT_DOCSTORE``: Este agente usa o framework ReAct para interagir com um repositório de documentos. Ele requer duas ferramentas: uma ferramenta de pesquisa para procurar um documento e uma ferramenta de consulta para procurar um termo no documento mais recente encontrado.

- ``SELF_ASK_WITH_SEARCH``: Este agente usa uma única ferramenta chamada "Intermediate Answer" para procurar respostas factuais para perguntas. Ele é equivalente ao artigo original "self-ask with search", onde uma API de pesquisa do Google foi usada como ferramenta.

- ``STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION``: Este agente é otimizado para modelos de chat e pode usar ferramentas com várias entradas. Ele usa o framework ReAct e é capaz de criar uma entrada de ação estruturada com base no esquema de argumentos da ferramenta.

- ``ZERO_SHOT_REACT_DESCRIPTION``: Este agente usa o framework ReAct para decidir qual ferramenta usar com base na descrição da ferramenta. Ele pode receber várias ferramentas como entrada e é o agente mais geral e versátil.

## Agent events

<center><img src="https://miro.medium.com/v2/resize:fit:1400/1*uEAfllPdUxZKEkiRIuZFdA.png" width="1500"></center>

- Considerando o diagrama abaixo, ao receber uma solicitação _(task)_, os Agentes aproveitam os LLM's para tomar uma decisão sobre qual ação tomar.
- Após a conclusão de uma _ação_ (action), o Agente entra na etapa de _observação_ (observation).
- Na etapa de _observação_, o Agente compartilha um _pensamento_ (thought), se uma resposta final não for alcançada,
- O Agente volta para outra _ação_ para se aproximar de uma _resposta final_.
- É possível _setar_ a quantidade máxima de iterações que o Agent pode recorrer para encontrar a _resposta final_ (max_iterations)

# Build agent

### Create custom tools from functions

In [ ]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """
    Multiply two integers together and return the result.

    Parameters:
    first_int (int): The first integer to be multiplied.
    second_int (int): The second integer to be multiplied.

    Returns:
    int: The product of the two input integers.
    """
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    """
    Add two integers together and return the result.

    Parameters:
    first_int (int): The first integer to be added.
    second_int (int): The second integer to be added.

    Returns:
    int: The sum of the two input integers.
    """
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    """
    Raise the base to the power of the exponent and return the result.

    Parameters:
    base (int): The base number.
    exponent (int): The exponent to which the base is raised.

    Returns:
    int: The result of raising the base to the power of the exponent.
    """
    return base**exponent

# define a list of tools for further use
tools = [multiply, add, exponentiate]

### Create prompt

In [ ]:
# get the prompt to use
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()


# read more about langchain hub: https://docs.smith.langchain.com/cookbook/hub-examples 

In [ ]:
agent = create_openai_tools_agent(model, tools, prompt)

In [ ]:
# create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

- o ``AgentExecutor`` é uma classe na LangChain que atua como um agente que utiliza ferramentas para executar tarefas. 
- ele é responsável por criar um modelo válido a partir de dados de entrada, lidar com paradas precoces, e executar ações com base em um plano determinado pelo agente. 
- o ``AgentExecutor`` também suporta callbacks para manipular eventos durante a execução da cadeia. 

In [ ]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)

# pegue 3 elevado à quinta potência
# multiplique pela soma de doze e três
# eleve ao quadrado o resultado total

# Memory

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("hi!")

demo_ephemeral_chat_history.add_ai_message("whats up?")

- ``ChatMessageHistory()``: é uma classe na LangChain que serve como um invólucro leve para salvar mensagens de usuário e de IA, além de recuperá-las. 
    - pode ser usada diretamente para gerenciar a memória fora de uma cadeia

- ``add_user_message(message)`` e ``add_ai_message(message)``: são métodos da classe ``ChatMessageHistory()`` que permitem adicionar mensagens de usuário e de IA, respectivamente, à memória. 
    - eles são métodos de conveniência para adicionar mensagens humanas e de IA à memória. 

In [ ]:
demo_ephemeral_chat_history.messages

# see all msgs

In [ ]:
demo_ephemeral_chat_history.add_user_message(
    "Translate this sentence from English to French: I love programming."
)

response = chain.invoke({"messages": demo_ephemeral_chat_history.messages})

response

In [ ]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate this sentence from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

# Refs
## Langchain docs 
- [LCEL](https://python.langchain.com/docs/expression_language/)

## Explore base
- [The Langchain Interface: Chains and Runnables](https://jordan-mungujakisa.medium.com/the-langchain-interface-chains-and-runnables-cd2f2cb6b4d6#:~:text=To%20make%20it%20easy%20to,chain%20on%20a%20single%20input)